In [3]:
'''
Ensure pdfminer is install
'''
try:
    import pdfminer
except ImportError:
    install_pdfminer = input('You do not have pdfminer install. Type Yes to install using pip. Other just click Enter')
    if install_pdfminer == 'Yes':
        import pip
        pip.main(['install','pdfminer'])

In [151]:
import pandas
import re
from cStringIO import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = file(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

def cleanUp(convertedString):
    convertedString = re.sub('\n','',convertedString)
    convertedString = re.sub(r'\[Crossref]','',convertedString)
    convertedString = re.sub(r'  ',' ',convertedString)
    return convertedString

def paperSplitter(convertedString):
    convertedString = convertedString.lower()
    portioned_paper = []
    
    if re.search('materials', convertedString):
        intString = re.split('materials', convertedString)
        portioned_paper.append(('Start', intString[0]))
    
    if re.search('materials and methods', convertedString):
        if re.search('materials and methods', convertedString) and re.search('results', convertedString):    
            intString = re.split('materials and methods', convertedString)
            intString = re.split('results', intString[-1])
            portioned_paper.append(('Materials & Methods', intString[0]))

    elif re.search('materials', convertedString):
        if re.search('materials', convertedString) and re.search('methods', convertedString):     
            intString = re.split('materials', convertedString)
            intString = re.split('methods', intString[-1])
            portioned_paper.append(('Materials', intString[0]))
                
        if re.search('methods', convertedString) and re.search('results', convertedString):    
            intString = re.split('methods', convertedString)
            intString = re.split('results', intString[-1])
            portioned_paper.append(('Methods', intString[0]))
    
    if re.search('results', convertedString) and re.search('discussion', convertedString):    
        intString = re.split('results', convertedString)
        intString = re.split('discussion', intString[-1])
        portioned_paper.append(('Results', intString[0]))
    
    if re.search('discussion', convertedString) and re.search('conclusions', convertedString):
        intString = re.split('discussion', convertedString)
        intString = re.split('conclusions', intString[-1])
        portioned_paper.append(('Discussion', intString[0]))
        convertedString = convertedString[1]
    
    if re.search('conclusions', convertedString) and re.search('acknowledgments', convertedString):
        intString = re.split('conclusions', convertedString)
        intString = re.split('acknowledgments', intString[-1])
        portioned_paper.append(('Conclusions', intString[0]))
        convertedString = convertedString[1]
    
    if re.search('acknowledgments', convertedString):
        intString = re.split('acknowledgments', convertedString)
        portioned_paper.append(('Acknowledgments', intString[1]))
    return portioned_paper

In [145]:
pdf_as_string = cleanUp(convert('Papers/cjps81-035.pdf'))

In [146]:
re.search('materials and methods',pdf_as_string.lower())

In [152]:
paperSplitter(pdf_as_string)

[('Start',
  "effect of salinity on rhizobium-root.hairinteraction, nodulation and growth of soybeanj. c. turesearch station, agriculture canada, harrow, ontario n2r lg0. received1980, aacepted 20 oct. 1980.ll julytu' j. c. 198 i . effect of salinity on rhizobium-root-hair interaction, nodulationand growth ofsoybean. can. j. plant sci. 6l 231-239.the interaction of rhizobia with root-hairs in the rhizosphere of soybean wasinvestigated under saline concentrations of 0-1.87c. at loto nacl. roorhairsshowed little curling and deformation in response to rhizobial inoculation. at l.5vo orabove, shrinkage of root-hairs was cvident. the growth and multiplication of rhizobiadeclined rapidly when nacl concentrations were increased from 0.2 to 0.87o. athigher concentrations, rhizobial growth .vas minimal. nodulation was completelyeliminated at concentratio ns of 1 .2vc nacl or higher and increasing salinity caused igradual continuous decline in soybean fresh weight and plant height. the failure o

In [153]:
for i in range(0,len(paperSplitter(pdf_as_string))):
    print paperSplitter(pdf_as_string)[i][0]

Start
Materials & Methods
Results
Acknowledgments
